In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pytorchcv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 532 kB 31.7 MB/s 


In [ ]:
import cv2
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.transforms import ToTensor
from PIL import Image
import numpy as np
from torchvision.models import vgg16
import matplotlib.pyplot as plt
import random
import torch.nn as nn
import math 
from scipy.spatial.distance import euclidean
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.transforms import ToPILImage
from pytorchcv.model_provider import get_model# as ptcv_get_model

In [ ]:
def alignment_procedure(img, left_eye, right_eye):
    #this function aligns given face in img based on left and right eye coordinates
    
    left_eye_x, left_eye_y = left_eye
    right_eye_x, right_eye_y = right_eye
    
    #-----------------------
    #find rotation direction
    
    if left_eye_y > right_eye_y:
        point_3rd = (right_eye_x, left_eye_y)
        direction = -1 #rotate same direction to clock
    else:
        point_3rd = (left_eye_x, right_eye_y)
        direction = 1 #rotate inverse direction of clock
    
    #-----------------------
    #find length of triangle edges
    
    a = euclidean(left_eye, point_3rd)
    b = euclidean(right_eye, point_3rd)
    c = euclidean(right_eye, left_eye)
    
    #-----------------------
    
    #apply cosine rule
    
    if b != 0 and c != 0: #this multiplication causes division by zero in cos_a calculation
    
        cos_a = (b*b + c*c - a*a)/(2*b*c)
        angle = np.arccos(cos_a) #angle in radian
        angle = (angle * 180) / math.pi #radian to degree
    
        #-----------------------
        #rotate base image
    
        if direction == -1:
            angle = 90 - angle
    
        img = Image.fromarray(img)
        img = np.array(img.rotate(direction * angle))
    
    #-----------------------
    
    return img #return img anyway


def get_references(features, age: int, df: pd.DataFrame, scheme: str = 'geo', r: float = 0.1, path='')->tuple():
  min_age = min(df['age'])
  max_age = max(df['age'])
  if scheme == 'geo':
    c = abs(1 - np.exp(2 * r)) * age / (1 + np.exp(2 * r))
    y1 = round(age - c)
    y2 = round(age + c)
    # y1 = random.randint(3, 25)
    # y2 = random.randint(26, 80)
    # while y2 - y1 <= 3:
    #   y1 = random.randint(3, 25)
    #   y2 = random.randint(26, 80)
  else:
    y1 = int(age - r)
    y2 = int(age + r)
  if y1 < min_age:
    y1 = min_age
  if y2 > max_age:
    y2 = max_age
  # print(y1)
  # print('-'*10)
  # print(y2)
  df_y1 = df[df['age'] == y1]
  df_y2 = df[df['age'] == y2]
  while len(df_y1) == 0:
    #print(f'y1={y1}')
    y1 += 1
    df_y1 = df[df['age'] == y1]
  while len(df_y2) == 0:
    #print(f'y2={y2}')
    y2 -= 1
    df_y2 = df[df['age'] == y2]
  # if len(df_y1) == 1:
  #   output_y1 = features[df_y1.iloc[0].name]
  # else:
  #   row_y1 = random.randint(0, len(df_y1) - 1)
  #   output_y1 = features[df_y1.iloc[row_y1].name]

  # if len(df_y2) == 1:
  #   output_y2 = features[df_y2.iloc[0].name]
  # else:
  #   row_y2 = random.randint(0, len(df_y2) - 1)
  #   output_y2 = features[df_y2.iloc[row_y2].name]
  if len(df_y1) == 1:
    row_y1 = 0
  else:
    row_y1 = random.randint(0, len(df_y1) - 1)
  path_y1 = df_y1.iloc[row_y1, 1]
  output_y1, _ = prepare_img(path + path_y1, df_y1, row_y1)

  if len(df_y2) == 1:
    row_y2 = 0
  else:
    row_y2 = random.randint(0, len(df_y2) - 1)
  path_y2 = df_y2.iloc[row_y2, 1]
  output_y2, _ = prepare_img(path + path_y2, df_y2, row_y2)

  return output_y1, output_y2, np.log(y1), np.log(y2)

In [ ]:
def prepare_img(path, df, idx):
  #print(path)
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  label = int(df.iloc[idx, 2])
  # crop face
  bbox = df.iloc[idx, 3][1:-1]
  bbox = [int(x) for x in bbox.split(',')]

  face_cropped = img[bbox[1]: bbox[3], bbox[0]: bbox[2], :]
  # get eyes and align
  eye = df.iloc[idx, 4][1:-1]
  left_eye = [int(x) for x in eye.split(',')]
  left_eye = (left_eye[0], left_eye[1])

  eye = df.iloc[idx, 5][1:-1]
  right_eye = [int(x) for x in eye.split(',')]
  right_eye = (right_eye[0], right_eye[1])
  
  img = alignment_procedure(face_cropped, left_eye, right_eye)

  return img, label

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# model = vgg16(pretrained=True)
# return_nodes = [
#       "features"
#       ]
# model = create_feature_extractor(model, return_nodes=return_nodes)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, annotations_file, path):
      
    self.img_labels = pd.read_csv(annotations_file)
    self.transform = transforms.Compose([ ToPILImage(), transforms.Resize((224, 224)), ToTensor(),
                                          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                        ])
    self.path = path

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = str(self.img_labels.iloc[idx, 1])
    #img = Image.open(self.path + img_path)
    img = cv2.imread(self.path + img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    label = int(self.img_labels.iloc[idx, 2])
    # crop face
    bbox = self.img_labels.iloc[idx, 3][1:-1]
    bbox = [int(x) for x in bbox.split(',')]

    face_cropped = img[bbox[1]: bbox[3], bbox[0]: bbox[2], :]
    # get eyes and align
    eye = self.img_labels.iloc[idx, 4][1:-1]
    left_eye = [int(x) for x in eye.split(',')]
    left_eye = (left_eye[0], left_eye[1])

    eye = self.img_labels.iloc[idx, 5][1:-1]
    right_eye = [int(x) for x in eye.split(',')]
    right_eye = (right_eye[0], right_eye[1])
    
    img = alignment_procedure(face_cropped, left_eye, right_eye)
    img = self.transform(img)
    #img = img.unsqueeze(0)
        
    return img, label

class CustomDataset2(Dataset):
  def __init__(self, annotations_file, path, nns_df, features):
      
    self.img_labels = pd.read_csv(annotations_file)
    #self.transform = transforms.Compose([ ToPILImage(), transforms.Resize((224, 224)), ToTensor(),
    #                                     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #                                    ])
    self.path = path
    self.features = features
    self.nns_df = nns_df

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    x = torch.tensor(self.features[idx], device=device, dtype=torch.float)

    p_age = int(self.nns_df.iloc[idx, 0])
    y1, y2, theta_y1, theta_y2 = get_references(self.features, p_age, self.img_labels)
    
    label = int(self.img_labels.iloc[idx, 2])
    mu = (theta_y1 + theta_y2) / 2
    r = (theta_y2 - theta_y1) / 2
    if np.log(label) <= theta_y1:
      p_rank_gt = -1
    elif np.log(label) >= theta_y2:
      p_rank_gt = 1
    else:
      p_rank_gt = (np.log(label) - mu) / (r + 1e-10)
    y1 = torch.tensor(y1, device=device, dtype=torch.float)
    y2 = torch.tensor(y2, device=device, dtype=torch.float)

    return y1.resize_(y1.shape[0], 1, 1), x.resize_(x.shape[0], 1, 1), y2.resize_(y2.shape[0], 1, 1), label, p_rank_gt, mu, r


class CustomDataset3(Dataset):
  def __init__(self, annotations_file, path, nns_df, features):
      
    self.img_labels = pd.read_csv(annotations_file)
    self.transform = transforms.Compose([ ToPILImage(), transforms.Resize((224, 224)), ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                       ])
    self.path = path
    self.features = features
    self.nns_df = nns_df

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    # prepare x
    img_path = str(self.img_labels.iloc[idx, 1])
    
    x, theta_x = prepare_img(self.path + img_path, self.img_labels, idx)
    
    x = self.transform(x)

    # prepare y1, y2
    p_age = int(self.nns_df.iloc[idx, 0])
    y1, y2, theta_y1, theta_y2 = get_references(self.features, p_age, self.img_labels, path = self.path)

    mu = (theta_y1 + theta_y2) / 2
    r = (theta_y2 - theta_y1) / 2

    if np.log(theta_x) <= theta_y1:
      p_rank_gt = -1
    elif np.log(theta_x) >= theta_y2:
      p_rank_gt = 1
    else:
      p_rank_gt = (np.log(theta_x) - mu) / (r + 1e-10)

    y1 = self.transform(y1)
    y2 = self.transform(y2)

    return y1, x, y2, theta_x, p_rank_gt, mu, r

In [ ]:
df_nns_train = pd.read_csv('nns_train_train_after_encoder_checkpoint_v2.csv')#pd.read_csv('nns_train_vgg16.csv')
df_nns_val = pd.read_csv('nns_val_train_after_encoder_checkpoint_v2.csv')#pd.read_csv('nns_val_vgg16.csv')
df_nns_test = pd.read_csv('nns_test_train_after_encoder_checkpoint_v2.csv')#pd.read_csv('nns_test_vgg16.csv')


df_nns_test = df_nns_test.drop(labels='Unnamed: 0', axis=1)
df_nns_test['nns'] = df_nns_test['theta']
df_nns_test = df_nns_test.drop(labels='theta', axis=1)


df_nns_train = df_nns_train.drop(labels='Unnamed: 0', axis=1)
df_nns_train['nns'] = df_nns_train['theta']
df_nns_train = df_nns_train.drop(labels='theta', axis=1)


df_nns_val = df_nns_val.drop(labels='Unnamed: 0', axis=1)
df_nns_val['nns'] = df_nns_val['theta']
df_nns_val = df_nns_val.drop(labels='theta', axis=1)

In [ ]:
train_features = np.load('vgg16_train.npy')
val_features = np.load('vgg16_val.npy')
test_features = np.load('vgg16_test.npy')

In [ ]:
# ds_train = CustomDataset('train.csv', '/content/drive/MyDrive/AgeEstimation/CLAP/Train/Train/')
# ds_test = CustomDataset('test.csv', '/content/drive/MyDrive/AgeEstimation/CLAP/Test/Test/Test/')
# ds_val = CustomDataset('val.csv', '/content/drive/MyDrive/AgeEstimation/CLAP/Val/Validation/')

ds_train = CustomDataset3('train.csv', '/content/drive/MyDrive/AgeEstimation/CLAP/Train/Train/', df_nns_train, train_features)
ds_test = CustomDataset3('test.csv', '/content/drive/MyDrive/AgeEstimation/CLAP/Test/Test/Test/', df_nns_test, test_features)
ds_val = CustomDataset3('val.csv', '/content/drive/MyDrive/AgeEstimation/CLAP/Val/Validation/', df_nns_val, val_features)

In [ ]:
class Regressor(nn.Sequential):
    def __init__(self, input_channel, output_channel):
      super(Regressor, self).__init__()
      self.convA = nn.Conv2d(input_channel, output_channel, kernel_size=1, stride=1)
      self.leakyreluA = nn.ReLU()
      self.convB = nn.Conv2d(output_channel, output_channel, kernel_size=1, stride=1)
      self.leakyreluB = nn.ReLU()
      self.dropout = nn.Dropout(p=0.5)
      self.convC = nn.Conv2d(output_channel, 1, kernel_size=1, stride=1)
      self.activation = nn.Tanh()


    def forward(self, x):
      x = self.convA(x)
      x = self.leakyreluA(x)
      x = self.convB(x)
      x = self.leakyreluB(x)
      x = self.dropout(x)
      x = self.convC(x)

      return self.activation(x)

In [ ]:
class Global_Regressor(nn.Module):
    def __init__(self):
      super(Global_Regressor, self).__init__()
      #model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
      # model = vgg16(pretrained=True)
      # return_nodes = [
      # "features"
      # ]
      # model = create_feature_extractor(model, return_nodes=return_nodes)
      self.encoder = get_model("bn_vgg16", pretrained=True)
      #self.encoder = model
      self.avg_pool = nn.AvgPool2d(kernel_size=7)
      self.regressor = Regressor(1536, 512)

    def forward_siamese(self, x):
      x = self.encoder.features.stage1(x)
      x = self.encoder.features.stage2(x)
      x = self.encoder.features.stage3(x)
      x = self.encoder.features.stage4(x)
      x = self.encoder.features.stage5(x)
      x = self.avg_pool(x)

      return x

    def forward(self, y1, x, y2, stage='train'):
      
      if stage == 'etc':
        x = torch.cat([y1, x, y2], dim=1)

        output = self.regressor(x)
      else:
        y1 = self.forward_siamese(y1)
        x = self.forward_siamese(x)
        y2 = self.forward_siamese(y2)

        x = torch.cat([y1, x, y2], dim=1)

        output = self.regressor(x)

      return output

      

In [ ]:
class Global_Regressor2(nn.Module):
  def __init__(self):
    super(Global_Regressor2, self).__init__()
    #model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
    # model = vgg16(pretrained=True)
    return_nodes = [
    "features"
    ]
    # model = create_feature_extractor(model, return_nodes=return_nodes)
    self.encoder = create_feature_extractor(get_model("bn_vgg16", pretrained=True), return_nodes=return_nodes)
    #self.encoder = model
    self.avg_pool = nn.AvgPool2d(kernel_size=7)
    self.regressor = Regressor(1536, 512)

  def forward_siamese(self, x):
    x = self.encoder(x)['features']
    # x = self.encoder.features.stage2(x)
    # x = self.encoder.features.stage3(x)
    # x = self.encoder.features.stage4(x)
    # x = self.encoder.features.stage5(x)
    x = self.avg_pool(x)

    return x

  def forward(self, y1, x, y2, stage='train'):
    if stage == 'etc':
      x = torch.cat([y1, x, y2], dim=1)

      output = self.regressor(x)
    else:
      y1 = self.forward_siamese(y1)
      x = self.forward_siamese(x)
      y2 = self.forward_siamese(y2)

      x = torch.cat([y1, x, y2], dim=1)

      output = self.regressor(x)

    return output

In [ ]:
model = Global_Regressor2()
model = model.to(device)

In [ ]:
model.load_state_dict(torch.load('drive/MyDrive/model_MWR_global.pth'))

<All keys matched successfully>

In [ ]:
batch_size = 32

In [ ]:
train_loader = DataLoader(ds_train, batch_size=batch_size, shuffle=False, num_workers=1)
val_loader = DataLoader(ds_val, batch_size=batch_size, shuffle=False, num_workers=1)
test_loader = DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=1)

In [ ]:
# model = vgg16(pretrained=True)
# return_nodes = [
#       "features"
#       ]
# model = create_feature_extractor(model, return_nodes=return_nodes)

In [ ]:
# train_features = []
# val_features = []
# test_features = []
# pool = nn.AvgPool2d(kernel_size=7)


# for step,batch in enumerate(train_loader):
#   if step % 50 == 0 and not step == 0:
#     print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_loader)))
#   images, labels = batch
#   with torch.no_grad():
#     features = model(images)['features']
#     features = pool(features)
#   for i in range(len(features)):
#     b = features[i].squeeze().detach().cpu().numpy()
#     train_features.append(b)



# for step,batch in enumerate(val_loader):
#   if step % 10 == 0 and not step == 0:
#     print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_loader)))
#   images, labels = batch
#   with torch.no_grad():
#     features = model(images)['features']
#     features = pool(features)
#   for i in range(len(features)):
#     b = features[i].squeeze().detach().cpu().numpy()
#     val_features.append(b)


# for step,batch in enumerate(test_loader):
#   if step % 10 == 0 and not step == 0:
#     print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_loader)))
#   images, labels = batch
#   with torch.no_grad():
#     features = model(images)['features']
#     features = pool(features)
#   for i in range(len(features)):
#     b = features[i].squeeze().detach().cpu().numpy()
#     test_features.append(b)

In [ ]:
# train_features = np.array(train_features)
# val_features = np.array(val_features)
# test_features = np.array(test_features)

In [ ]:
# np.save('vgg16_train.npy', train_features)
# np.save('vgg16_val.npy', val_features)
# np.save('vgg16_test.npy', test_features)

In [ ]:
# df_train = pd.read_csv('train.csv')
# df_val = pd.read_csv('val.csv')
# df_test = pd.read_csv('test.csv')
# df_train.head()

In [ ]:
# from sklearn.metrics import pairwise_distances

# train_ages = list(df_train['age'])
# val_ages = list(df_val['age'])
# test_ages = list(df_test['age'])

# train_theta = []
# for i, e in enumerate(train_features):
#   e = e.reshape(1, -1)
#   dists = []
#   for j, e2 in enumerate(train_features):
#     e2 = e2.reshape(1, -1)
#     dists.append(pairwise_distances(e, e2))
  
#   ages = train_ages.copy()

#   dists.pop(i)
#   ages.pop(i)
  
#   dists, ages = zip(*sorted(zip(dists, ages), reverse=True))
#   p = sum(ages[:5]) / 5
#   train_theta.append(p)

# val_theta = []
# for i, e in enumerate(val_features):
#   e = e.reshape(1, -1)
#   dists = []
#   for j, e2 in enumerate(val_features):
#     e2= e2.reshape(1, -1)
#     dists.append(pairwise_distances(e, e2))
  
#   ages = val_ages.copy()

#   dists.pop(i)
#   ages.pop(i)
  
#   dists, ages = zip(*sorted(zip(dists, ages), reverse=True))
#   p = sum(ages[:5]) / 5
#   val_theta.append(p)


# test_theta = []
# for i, e in enumerate(test_features):
#   e = e.reshape(1, -1)
#   dists = []
#   for j, e2 in enumerate(test_features):
#     e2 = e2.reshape(1, -1)
#     dists.append(pairwise_distances(e, e2))
  
#   ages = test_ages.copy()

#   dists.pop(i)
#   ages.pop(i)
  
#   dists, ages = zip(*sorted(zip(dists, ages), reverse=True))
#   p = sum(ages[:5]) / 5
#   test_theta.append(p)

In [ ]:
# df_nns_train = pd.DataFrame(data={'nns': train_theta})
# df_nns_val = pd.DataFrame(data={'nns': val_theta})
# df_nns_test = pd.DataFrame(data={'nns': test_theta})

In [ ]:
# df_nns_train.to_csv('nns_train_vgg16.csv', index=False)
# df_nns_val.to_csv('nns_val_vgg16.csv', index=False)
# df_nns_test.to_csv('nns_test_vgg16.csv', index=False)

In [ ]:
optimizer = optim.AdamW(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=2,verbose=True)
criterion = nn.MSELoss()

In [ ]:
epochs = 300
df_test = pd.read_csv('test.csv')

In [ ]:
def train():
  
  model.train()

  total_loss = 0.0

  total_preds=[]

  for step,batch in enumerate(train_loader):
    #if step % 50 == 0 and not step == 0:
    print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_loader)))
    batch = [r.to(device) for r in batch]
    
    y1, x, y2, labels, p_rank_gts, _, _ = batch
    p_rank_gts = torch.tensor(p_rank_gts, device=device, dtype=torch.float)
    model.zero_grad() 
    preds = model(y1, x, y2).squeeze()
    loss = criterion(preds, p_rank_gts)
    total_loss = total_loss + loss.item()
    loss.backward()
    optimizer.step()
    preds=preds.detach().cpu().numpy()
    total_preds.append(preds)

  avg_loss = total_loss / len(train_loader)

  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
transform = transforms.Compose([ ToPILImage(), transforms.Resize((224, 224)), ToTensor(),
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                       ])

def evaluate(loader, stage='val'):
  
  print("\nEvaluating...")

  model.eval()

  total_loss = 0.0

  total_preds = []
  d = {}
  for step,batch in enumerate(loader):
  
    #if step % 50 == 0 and not step == 0:
              
    print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(loader)))

    batch = [t.to(device) for t in batch]
    
    #faces, fulls, labels = batch
    y1, x, y2, labels, p_rank_gts, mu, r = batch
    p_rank_gts = torch.tensor(p_rank_gts, device=device, dtype=torch.float)
    #r = r.detach().cpu().numpy()
    #mu = r.detach().cpu().numpy()
    with torch.no_grad():
  
      #preds = model(faces, fulls)
      preds = model(y1, x, y2).squeeze()

      loss = criterion(preds,p_rank_gts)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()
      thetas = []
      for p, r_curr, mu_curr in zip(preds, r, mu):
        elem = np.exp(p * r_curr.detach().cpu().item() + mu_curr.detach().cpu().item())
        thetas.append(elem)
      if 0 not in d.keys():
        d[0] = []
      if stage != 'val':
        d[0].append(thetas)
        for i in range(4):
          y1s = []
          y2s = []
          mus = []
          rs = []
          for e in thetas:
            
            y1, y2, theta_y1, theta_y2 = get_references(test_features, e, df_test, path='/content/drive/MyDrive/AgeEstimation/CLAP/Test/Test/Test/')
            
            mu_i = (theta_y1 + theta_y2) / 2
            r_i = (theta_y2 - theta_y1) / 2
            y1s.append(transform(y1))
            y2s.append(transform(y2)) 
            mus.append(mu_i) 
            rs.append(r_i) 
          #y1s = np.array(y1s)
          #y2s = np.array(y2s)
          # y1 = transform(y1s)
          # y2 = transform(y2s)
          y1 = torch.stack(y1s).to(device)#.resize_(y1s.shape[0], y1s.shape[1], 1, 1)
          y2 = torch.stack(y2s).to(device)#.resize_(y2s.shape[0], y2s.shape[1], 1, 1)
          preds = model(y1, x, y2).squeeze()
          preds = preds.detach().cpu().numpy()

          thetas = []
          for p, r_curr, mu_curr in zip(preds, rs, mus):
            thetas.append(np.exp(p * r_curr + mu_curr))
          if i + 1 not in d.keys():
            d[i + 1] = []
          
          d[i + 1].append(thetas)

      total_preds.append(thetas)

  avg_loss = total_loss / len(loader) 

  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds, d

In [ ]:
from sklearn.metrics import mean_absolute_error
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]
test_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    train_loss, _ = train()

    valid_loss, _, _ = evaluate(val_loader)
    scheduler.step(valid_loss)

    if valid_loss < best_valid_loss:
       best_valid_loss = valid_loss
        
    #torch.save(model.state_dict(), 'drive/MyDrive/MWR_CLAP' + str(epoch) + '.pt')

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    
    # with open('drive/MyDrive/train_loss.txt', 'a') as f:
    #     f.write(str(train_loss) + '\n')
        
        
    # with open('drive/MyDrive/valid_loss.txt', 'a') as f:
    #     f.write(str(valid_loss) + '\n')

        
    test_loss, preds, list_preds = evaluate(test_loader, stage='test')
    test_losses.append(test_loss)
    
    
    # with open('drive/MyDrive/test_loss.txt', 'a') as f:
    #     f.write(str(test_loss) + '\n')
        

    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')

    for i in range(len(list_preds.keys())):
      
      MAE = mean_absolute_error(list(df_test['age']), np.concatenate(list_preds[i], axis=0))
      print(f'MAE on MWR with {i + 1} steps = {MAE}')

    print(f'\nMAE on test-set: {MAE:.3f}')

print(f'\nBest valid loss: {best_valid_loss:.3f}')